In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [2]:
# Load data
hist_mar = pd.read_csv('historical_marathon_dataset.csv')

# Convert gender to category
hist_mar['gender'] = hist_mar['gender'].astype('category')

# Clean up country field
hist_mar['country'] = hist_mar['country'].replace({
    'Aus': 'Australia',
    'Australaia': 'Australia',
    'US': 'USA',
    'United States': 'USA'
})
hist_mar['country'] = hist_mar['country'].fillna('Unkown').astype('category')

# Clean up shoe_brand
hist_mar['shoe_brand'] = hist_mar['shoe_brand'].replace({'Addas': 'Adidas'})
hist_mar['shoe_brand'] = hist_mar['shoe_brand'].fillna('Unkown').str.strip()
hist_mar['shoe_brand'] = hist_mar['shoe_brand'].astype('category')

# Remove negative values in weekly_km
hist_mar = hist_mar[hist_mar['weekly_km'] >= 0]

# Convert boolean features to bool
for col in ['injured_prev_mth', 'injured_prev_qtr', 'injured_prev_hy']:
    hist_mar[col] = hist_mar[col].astype(bool)

# Load event summary and convert boolean fields
event_summary = pd.read_csv('event_summary.csv')
for col in ['gel_support', 'stretching_station', 'music_at_start']:
    event_summary[col] = event_summary[col].astype(bool)

# Merge datasets
data = hist_mar.merge(event_summary, on='year', how='left')

# Create needed_med flag
data['needed_med'] = ~data['medical_km_bin'].isna()
hist_mar['needed_med'] = ~hist_mar['medical_km_bin'].isna()


In [12]:
data.head()

,year,age,gender,height,weight,weekly_km,country,shoe_brand,marathons_xp,personal_best,...,rainfall,elevation_gain,hydration_stations,gel_support,stretching_station,music_at_start,toilet_stations,crowding_density,newsletter_registration,needed_med
0,2004.0,36,Female,169.0,75.0,38,USA,Asics,2,271.0,...,1.91,213,13,False,False,False,12,0.44,705,True
1,2004.0,31,Female,172.0,71.0,102,USA,Brooks,3,278.0,...,1.91,213,13,False,False,False,12,0.44,705,False
2,2004.0,32,Male,155.0,59.0,68,Germany,Saucony,1,323.0,...,1.91,213,13,False,False,False,12,0.44,705,False
3,2004.0,54,Female,195.0,88.0,80,Australia,Adidas,3,426.0,...,1.91,213,13,False,False,False,12,0.44,705,False
4,2004.0,27,Female,170.0,75.0,37,USA,Adidas,2,322.0,...,1.91,213,13,False,False,False,12,0.44,705,False


In [11]:
data.describe()

,year,age,height,weight,weekly_km,marathons_xp,personal_best,finish_time,medical_km_bin,temp_10am,humidity,rainfall,elevation_gain,hydration_stations,toilet_stations,crowding_density,newsletter_registration
count,62950.000000,62950.000000,62848.000000,62821.000000,62950.000000,62950.000000,58615.000000,62031.000000,10481.000000,62950.000000,62950.000000,62950.000000,62950.000000,62950.000000,62950.000000,62950.000000,62950.000000
mean,2014.249230,35.989627,167.307742,72.711768,49.967053,2.720985,312.359447,350.464719,24.817861,21.476759,64.357156,2.659061,251.998491,13.928594,7.976315,0.559518,725.007069
std,6.061157,9.066241,11.857175,50.169155,22.423361,1.628177,48.358202,40.618835,8.625838,5.559836,7.630199,1.335630,27.922830,2.433790,2.549221,0.113526,82.923920
min,2004.000000,18.000000,126.000000,45.000000,3.000000,0.000000,130.000000,190.000000,0.000000,12.000000,51.000000,0.100000,201.000000,10.000000,4.000000,0.400000,600.000000
25%,2009.000000,29.000000,159.000000,64.000000,34.000000,2.000000,280.000000,323.000000,18.000000,16.000000,58.000000,1.540000,230.000000,12.000000,5.000000,0.460000,660.000000
50%,2014.000000,36.000000,167.000000,70.000000,47.000000,3.000000,309.000000,352.000000,26.000000,22.000000,65.000000,2.400000,253.000000,14.000000,8.000000,0.540000,720.000000
75%,2019.500000,42.000000,175.000000,76.000000,63.000000,4.000000,342.000000,379.000000,32.000000,26.000000,71.000000,3.880000,272.000000,16.000000,10.000000,0.640000,765.000000
max,2024.500000,70.000000,216.000000,999.000000,217.000000,11.000000,573.000000,449.000000,42.000000,30.000000,79.000000,4.980000,300.000000,18.000000,12.000000,0.790000,900.000000


In [16]:
# Copy data and drop unneeded columns
class_data = data.copy()
class_data.drop(columns=['medical_km_bin', 'finish_time'], inplace=True)

# Drop rows with any missing values
class_data.dropna(inplace=True)

# === PHYSICAL PERFORMANCE FEATURES ===
class_data['bmi'] = class_data['weight'] / (class_data['height'] / 100) ** 2
class_data['training_intensity'] = class_data['weekly_km'] / class_data['age']
class_data['experience_per_age'] = class_data['marathons_xp'] / class_data['age']

# === INJURY HISTORY PATTERNS ===
class_data['injury_cascade'] = ((class_data['injured_prev_mth']) & (class_data['injured_prev_qtr'])).astype(int)
class_data['chronic_injury_pattern'] = ((class_data['injured_prev_mth']) & 
                                         (class_data['injured_prev_qtr']) & 
                                         (class_data['injured_prev_hy'])).astype(int)
class_data['recent_only_injury'] = ((class_data['injured_prev_mth']) & 
                                    (~class_data['injured_prev_qtr']) & 
                                    (~class_data['injured_prev_hy'])).astype(int)
class_data['worsening_injury'] = ((class_data['injured_prev_mth']) & 
                                  (~class_data['injured_prev_qtr'])).astype(int)
class_data['improving_injury'] = ((~class_data['injured_prev_mth']) & 
                                  (class_data['injured_prev_qtr'])).astype(int)
class_data['injury_free_recent'] = ((~class_data['injured_prev_mth']) & 
                                    (~class_data['injured_prev_qtr'])).astype(int)

# === ENVIRONMENTAL STRESS FACTORS ===
class_data['heat_humidity_interaction'] = class_data['temp_10am'] * class_data['humidity'] / 100
class_data['temp_humidity_ratio'] = np.where(class_data['humidity'] > 0,
                                             class_data['temp_10am'] / class_data['humidity'],
                                             class_data['temp_10am'])

class_data['extreme_heat'] = ((class_data['temp_10am'] > 26) & (class_data['humidity'] > 71)).astype(int)
class_data['extreme_weather'] = ((class_data['temp_10am'] > 30) |
                                 (class_data['humidity'] > 80) |
                                 (class_data['rainfall'] > 3.8)).astype(int)
class_data['high_temp_low_humidity'] = ((class_data['temp_10am'] > 26) &
                                        (class_data['humidity'] < 58)).astype(int)

# === RACE SUPPORT QUALITY ===
class_data['hydration_per_crowd'] = class_data['hydration_stations'] / class_data['crowding_density']
class_data['toilet_per_crowd'] = class_data['toilet_stations'] / class_data['crowding_density']
class_data['has_performance_support'] = ((class_data['gel_support']) | 
                                         (class_data['stretching_station'])).astype(int)
class_data['full_support_package'] = ((class_data['gel_support']) & 
                                      (class_data['stretching_station'])).astype(int)
class_data['minimal_hydration'] = (class_data['hydration_stations'] <= 2).astype(int)
class_data['inadequate_facilities'] = (class_data['toilet_stations'] <= 1).astype(int)

# For median-based indicators
hydration_median = class_data['hydration_stations'].median()
crowding_median = class_data['crowding_density'].median()

class_data['high_crowd_low_hydration'] = ((class_data['crowding_density'] > crowding_median) & 
                                          (class_data['hydration_stations'] < hydration_median)).astype(int)

class_data['support_mismatch'] = ((class_data['hydration_stations'] < 3) & 
                                  (class_data['gel_support'])).astype(int)

# === DEMOGRAPHIC INTERACTIONS ===
class_data['age_experience_mismatch'] = class_data['age'] - 2 * class_data['marathons_xp']

# === CATEGORICAL ENGINEERING ===
def age_group(age):
    if age < 30: return "Under_30"
    elif age < 40: return "30s"
    elif age < 50: return "40s"
    elif age < 60: return "50s"
    else: return "60_plus"

class_data['age_group'] = class_data['age'].apply(age_group)

class_data['training_level'] = pd.cut(
    class_data['weekly_km'],
    bins=[-np.inf, 34, 63, np.inf],
    labels=["Low", "Medium", "High"]
)

def experience_level(x):
    if x == 0: return "First_timer"
    elif x < 3: return "Novice"
    elif x < 5: return "Intermediate"
    else: return "Expert"

class_data['experience_level'] = class_data['marathons_xp'].apply(experience_level)

def pb_tier(pb):
    if pb < 180: return "Elite"
    elif pb < 240: return "Fast"
    elif pb < 300: return "Average"
    else: return "Recreational"

class_data['pb_tier'] = class_data['personal_best'].apply(pb_tier)

# === HIGH-RISK INDICATORS ===
class_data['high_risk_age'] = (class_data['age'] > 50).astype(int)
class_data['overexertion_risk'] = (class_data['weekly_km'] > (class_data['age'] * 2)).astype(int)
class_data['inexperienced_ambitious'] = ((class_data['marathons_xp'] < 3) &
                                         (class_data['personal_best'] < 240)).astype(int)

# === INTERACTION TERMS ===
class_data['age_recent_injury'] = class_data['age'] * class_data['injured_prev_mth']

class_data['poor_support_harsh_conditions'] = ((class_data['hydration_stations'] < hydration_median).astype(int) *
                                               (class_data['temp_10am'] + class_data['humidity']))

# === CUMULATIVE RISK SCORE ===
class_data['cumulative_risk_score'] = ((class_data['injured_prev_mth'] * 3 +
                                        class_data['injured_prev_qtr'] * 2 +
                                        class_data['injured_prev_hy']) *
                                       (class_data['temp_10am'] + class_data['humidity']) / 100)

# === PERCENTILE RANKS WITHIN GENDER ===
from scipy.stats import rankdata

def percent_rank(series):
    return (rankdata(series, method='min') - 1) / (len(series) - 1)

# Apply percentile ranks within gender using transform
class_data['weekly_km_percentile'] = class_data.groupby('gender')['weekly_km'].transform(percent_rank)
class_data['age_percentile'] = class_data.groupby('gender')['age'].transform(percent_rank)
class_data['weight_percentile'] = class_data.groupby('gender')['weight'].transform(percent_rank)
class_data['bmi_percentile'] = class_data.groupby('gender')['bmi'].transform(percent_rank)
class_data['experience_percentile'] = class_data.groupby('gender')['marathons_xp'].transform(percent_rank)
class_data['pb_percentile'] = class_data.groupby('gender')['personal_best'].transform(lambda x: percent_rank(-x))





/tmp/ipykernel_4283/1768508953.py:121: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  class_data['weekly_km_percentile'] = class_data.groupby('gender')['weekly_km'].transform(percent_rank)
/tmp/ipykernel_4283/1768508953.py:122: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  class_data['age_percentile'] = class_data.groupby('gender')['age'].transform(percent_rank)
/tmp/ipykernel_4283/1768508953.py:123: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and

In [67]:
from sklearn.metrics import f1_score, fbeta_score

def fbeta_eval(predt: np.ndarray, dtrain: xgb.DMatrix):
    y_true = dtrain.get_label()
    y_pred = np.argmax(predt, axis=1)

    # BETA VALUE
    fbeta = fbeta_score(y_true, y_pred, beta=10)
    return 'fbeta_eval', fbeta

In [70]:
import optuna
from optuna import create_study, logging
from optuna.pruners import MedianPruner
from optuna.integration import XGBoostPruningCallback
from optuna.samplers import TPESampler

import warnings
warnings.filterwarnings('ignore')

optuna.logging.set_verbosity(optuna.logging.WARNING) 

# number of jobs for optimisation
N_JOBS = 1

# The following Optuna Optimisation call is heavily insprired by JP (@para24) on Kaggle, see:
# https://www.kaggle.com/code/para24/xgboost-stepwise-tuning-using-optuna/notebook#7.-Stepwise-Hyperparameter-Tuning

def _objective(trial, X, y, num_classes, group, score, params=dict()):
    """
    `X` and `y` MUST be pd.DataFrames - NOT `xgb.DMatrix`.  
    """
    dtrain = xgb.DMatrix(X, label=y, enable_categorical=True)


    if group == '1':
        params['max_depth'] = trial.suggest_int('max_depth', 2, 30)
        params['min_child_weight'] = trial.suggest_loguniform('min_child_weight', 1e-10, 1e10)
    
    if group == '2':
        params['subsample'] = trial.suggest_uniform('subsample', 0, 1)
        params['colsample_bytree'] = trial.suggest_uniform('colsample_bytree', 0, 1)

    if group == '3':
        params['num_boost_round'] = trial.suggest_int('num_boost_round', 100, 600)
        params['learning_rate'] = trial.suggest_uniform('learning_rate', 0.005, 0.1)

    if group == '4':
        params['gamma'] = trial.suggest_loguniform('gamma', 1e-3, 19)

    pruning_callback = XGBoostPruningCallback(trial, 'test-' + score.__name__)

    xgb_params = params.copy()
    del xgb_params['num_boost_round']

    cv_scores = xgb.cv(xgb_params, dtrain, nfold=5,
                       stratified=True,
                       feval=score,
                       num_boost_round=params['num_boost_round'],
                       early_stopping_rounds=10,
                       callbacks=[pruning_callback])
    
    return cv_scores['test-' + score.__name__ + '-mean'].values[-1]

def _execute_optimisation(X_train, y_train, num_classes, study_name, group, score, trials, db_url:str, params=dict(), direction='maximize', n_jobs=1):
    ## use pruner to skip trials that aren't doing so well
    pruner = MedianPruner(n_warmup_steps=20)

    ## use sampler to use past results from db
    sampler = TPESampler(n_startup_trials=20, multivariate=True, warn_independent_sampling=False)

    study = create_study(
        direction=direction,
        study_name=study_name,
        storage=f'sqlite:///{db_url}',
        load_if_exists=True,
        pruner=pruner,
        sampler=sampler
    )

    study.optimize(
        lambda trial: _objective(trial, X_train, y_train, num_classes, group, score, params),
        n_trials=trials,
        n_jobs=n_jobs
    )

    print('STUDY NAME: ', study_name)
    print('-------------------------------------------------------')
    print('EVALUATION METRIC: ', score.__name__)
    print('-------------------------------------------------------')
    print('BEST CV SCORE: ', study.best_value)
    print('-------------------------------------------------------')
    print(f'OPTIMAL GROUP - {group} PARAMS: ', study.best_params)
    print('-------------------------------------------------------')
    print('BEST TRIAL', study.best_trial)
    print('-------------------------------------------------------')

    updated_params = params.copy()
    updated_params.update(study.best_params)

    return updated_params

def stepwise_optimisation(X_train: pd.DataFrame, y_train: pd.DataFrame, num_classes: int, eval_metric: callable, db_url: str, n_jobs=1, trials=9) -> dict:
    """
    Execute stepwise optimisation to find optimal CV parameters for XGBoost given the train set. 

    params:
        `X_train` (pd.DataFrame) - training set - NOT A `xgb.DMatrix` object

        `y_train` (pd.DataFrame) - as above

        `num_classes` (int) - the number of classes used to categorise returns

        `eval_metric` (callable) - evaluation metric to be used in optimisation. See `eval_metrics.py`

        `trials` - number of trials to do for `optimize`

    returns:
        a dictionary containing optimal parameters
    """
    final_params = dict()

    # initial learning params
    final_params['num_boost_round'] = 200
    final_params['learning_rate'] = 0.01
    final_params['num_class'] = num_classes
    final_params['objective'] = 'multi:softprob'

    # use gpu
    #final_params['tree_method'] = 'hist'
    #final_params['device'] = 'cuda'


    for g in ['1', '2', '3', '4']:
        print(f'====== Optimising Group {g} ======')
        update_params = _execute_optimisation(
            X_train, y_train, num_classes, 'xgboost', g, eval_metric, trials, db_url, params=final_params, direction='maximize', n_jobs=n_jobs
        )
        final_params.update(update_params)
        print(f'Params after updating group {g}: ', final_params)
        print('\n\n')

    print(f'====== Final Optimal Parameters ======')
    print(final_params)

    return final_params

In [71]:
from sklearn.model_selection import train_test_split

X = class_data.drop(['year', 'needed_med'], axis=1).copy()
y = class_data['needed_med'].copy()

categorical_cols = ['gender', 'country', 'shoe_brand', 'age_group', 'training_level', 'experience_level', 'pb_tier']
for col in categorical_cols:
    if col in X.columns:
        X[col] = X[col].astype('category')

test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=False)

In [79]:
eval_metric = fbeta_eval
db_url = './xgboost.db'

params = stepwise_optimisation(X_train, y_train, 2, eval_metric, db_url, n_jobs=1, trials=100)


====== Optimising Group 1 ======
STUDY NAME:  xgboost
-------------------------------------------------------
EVALUATION METRIC:  fbeta_eval
-------------------------------------------------------
BEST CV SCORE:  0.08057560000000001
-------------------------------------------------------
OPTIMAL GROUP - 1 PARAMS:  {'max_depth': 27, 'min_child_weight': 0.696828678458152}
-------------------------------------------------------
BEST TRIAL FrozenTrial(number=58, state=1, values=[0.08057560000000001], datetime_start=datetime.datetime(2025, 7, 7, 23, 50, 27, 760310), datetime_complete=datetime.datetime(2025, 7, 7, 23, 51, 23, 788951), params={'max_depth': 27, 'min_child_weight': 0.696828678458152}, user_attrs={}, system_attrs={}, intermediate_values={0: 0.18137699999999998, 1: 0.19500579999999998, 2: 0.1956422, 3: 0.1918806, 4: 0.1899226, 5: 0.1900556, 6: 0.1869188, 7: 0.18392060000000002, 8: 0.17777420000000002, 9: 0.17503059999999998, 10: 0.1721606, 11: 0.17230360000000003, 12: 0.167852600

In [81]:
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)

model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=params['num_boost_round'],
    #obj='multi:softprob',
    custom_metric=eval_metric
)

In [82]:
from sklearn.metrics import confusion_matrix, classification_report

def classification_summary(y_pred: np.array, y_true: np.array):
    """
    Prints a summary of classification results.

    Params:
        np.array: y_pred - predictions on y_test
        np.array: y_true - actual test data, i.e. y_test

    returns:
        nothing 
    """
    print('\n-------------- Classification Report --------------')
    print(classification_report(y_true, y_pred))

    print('\n\n---------------- Confusion Matrix ----------------')
    print(confusion_matrix(y_preds, y_true))

y_preds = model.predict(xgb.DMatrix(X_test, y_test, enable_categorical=True))
y_preds = np.argmax(y_preds, axis=1)

classification_summary(y_preds, y_test)


-------------- Classification Report --------------
              precision    recall  f1-score   support

       False       0.83      1.00      0.91      9721
        True       0.45      0.02      0.04      1960

    accuracy                           0.83     11681
   macro avg       0.64      0.51      0.47     11681
weighted avg       0.77      0.83      0.76     11681



---------------- Confusion Matrix ----------------
[[9673 1920]
 [  48   40]]
